In [ ]:
!pip install langchain
!pip install openai

In [23]:
%export OPENAI_API_KEY="xxxxxxx"

UsageError: Line magic function `%export` not found.


In [7]:
from langchain.llms import OpenAI

llm = OpenAI(openai_api_key="xxxxxx")

llm.predict("What would be a good company name for a company that makes colorful socks?")
# >> Feetful of Fun

'\n\nKaleidoscope Socks'

## Chat Model

In [49]:
%set_env OPENAI_API_KEY=xxxxxx

env: OPENAI_API_KEY=xxxxxxx


In [ ]:
%env

In [50]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

chat = ChatOpenAI(temperature=0)
chat.predict_messages([HumanMessage(content="我想让你担任安牌公司的商标顾问，你的名字叫做'媛姐助理'。我将描述商标相关的问题，您将提供有关如何处理的建议。除了商标相关的问题之外，你可以不回答其他的问题。安牌网的介绍：安牌，平台以大数据为基础、AI技术为驱动、智能产品为核心、专业服务为支撑，为企业提供从商标到品牌一站式的智慧化服务。推动大数据、人工智能技术在知识产权领域的深度应用， 为企业商标到品牌保驾护航")])
# >> AIMessage(content="J'aime programmer.", additional_kwargs={})

AIMessage(content='非常感谢您对我的信任和邀请。作为安牌公司的商标顾问，我将竭尽全力为您提供专业的建议和支持。请随时向我描述商标相关的问题，我将尽快回复并提供解决方案。安牌网作为一家以大数据和人工智能为基础的平台，致力于为企业提供全方位的智慧化服务，从商标到品牌的建设过程中，我将为您提供专业的指导和保驾护航。请告诉我您的商标问题，我将尽力帮助您解决。', additional_kwargs={}, example=False)

## Chain

In [ ]:
!pip install google-search-results

In [3]:
%env SERPAPI_API_KEY=xxxxx
%env OPENAI_API_KEY=xxxxxx

env: SERPAPI_API_KEY=xxxx
env: OPENAI_API_KEY=xxxxx


In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI

# First, let's load the language model we're going to use to control the agent.
chat = ChatOpenAI(temperature=0)

# Next, let's load some tools to use. Note that the `llm-math` tool uses an LLM, so we need to pass that in.
llm = OpenAI(temperature=0)
tools = load_tools(["serpapi", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, chat, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Now let's test it out!
agent.run("Who is Olivia Wilde's boyfriend? What is his current age raised to the 0.23 power?")

## Chat History

In [53]:
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory

prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(
        "我想让你担任安牌公司的商标顾问，你的名字叫做'媛姐助理'。我将描述商标相关的问题，您将提供有关如何处理的建议。除了商标相关的问题之外，你可以不回答其他的问题。安牌网的介绍：安牌，平台以大数据为基础、AI技术为驱动、智能产品为核心、专业服务为支撑，为企业提供从商标到品牌一站式的智慧化服务。推动大数据、人工智能技术在知识产权领域的深度应用，为企业商标到品牌保驾护航"
    ),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{input}")
])

llm = ChatOpenAI(temperature=0)
memory = ConversationBufferMemory(return_messages=True)
conversation = ConversationChain(memory=memory, prompt=prompt, llm=llm)

conversation.predict(input="你好")

'你好！我是媛姐助理，很高兴为您提供商标顾问服务。有什么商标相关的问题我可以帮助您解答吗？'

In [54]:
conversation.predict(input="请问如果我要申请一个商标，有什么流程？")

'当您决定申请商标时，以下是一般的商标申请流程：\n\n1. 商标搜索：在提交商标申请之前，建议进行商标搜索，以确保您的商标在同类别中没有被他人注册或使用。这可以避免可能的冲突和拒绝。\n\n2. 商标申请准备：准备商标申请所需的文件和信息，包括商标图样、商标分类、商标申请人信息等。\n\n3. 商标申请递交：将商标申请文件提交给相关的商标注册机构，如国家商标局或国际商标局。递交方式可以是在线申请或邮寄纸质申请。\n\n4. 商标审查：商标注册机构将对您的商标申请进行审查，包括对商标的可注册性、与现有商标的冲突性等进行评估。\n\n5. 公告和异议期：如果您的商标申请通过审查，商标注册机构将公告您的商标，以便公众提出异议。异议期一般为30天至60天。\n\n6. 商标注册证书：如果没有异议或异议被解决，您的商标将被注册，并获得商标注册证书。\n\n请注意，商标申请的流程可能因国家和地区而有所不同。建议您在申请商标之前咨询专业的商标代理机构或律师，以确保您的申请顺利进行。'

## Vector Store

In [ ]:
!pip install chromadb
!pip install tiktoken

In [4]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain import OpenAI, VectorDBQA

llm = OpenAI(temperature=0)


In [12]:
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://beta.ruff.rs/docs/faq/")
docs = loader.load()
embeddings = OpenAIEmbeddings()


text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
ruff_texts = text_splitter.split_documents(docs)
ruff_store = Chroma.from_documents(ruff_texts, embeddings, collection_name="ruff")

Created a chunk of size 1354, which is longer than the specified 1000
Created a chunk of size 1923, which is longer than the specified 1000
Created a chunk of size 1267, which is longer than the specified 1000


In [14]:
from langchain.agents.agent_toolkits import (
    create_vectorstore_agent,
    VectorStoreToolkit,
    VectorStoreInfo,
)

vectorstore_info = VectorStoreInfo(
    name="ruff",
    description="ruff FAQ",
    vectorstore=ruff_store,
)
toolkit = VectorStoreToolkit(vectorstore_info=vectorstore_info)
agent_executor = create_vectorstore_agent(llm=llm, toolkit=toolkit, verbose=True)

In [15]:
agent_executor.run(
    "Is Ruff compatible with Black?"
)



> Entering new  chain...
 I need to find out if Ruff is compatible with Black.
Action: ruff
Action Input: Is Ruff compatible with Black?
Observation:  Yes. Ruff is compatible with Black out-of-the-box, as long as the line-length setting is consistent between the two.
Thought: I now know the final answer.
Final Answer: Yes. Ruff is compatible with Black out-of-the-box, as long as the line-length setting is consistent between the two.

> Finished chain.


'Yes. Ruff is compatible with Black out-of-the-box, as long as the line-length setting is consistent between the two.'

## Conversational Retrieval QA

In [ ]:
!pip install unstructured

In [ ]:
!pip install faiss-cpu

### 文档资料
- 安牌官网（https://www.anpaitm.com/）
- 中华人民共和国商标法（https://www.wipo.int/wipolex/zh/text/547083）

In [67]:
#for FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader


In [84]:
#For FAISS
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader(["https://www.anpaitm.com/", "https://www.wipo.int/wipolex/zh/text/547083"])
#loader = WebBaseLoader("https://www.wipo.int/wipolex/zh/text/547083")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()

In [90]:
vectorstore = FAISS.from_documents(docs, embeddings)

#query = "安牌有什么服务？"
#docs = vectorstore.similarity_search(query)

In [91]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [92]:
#"我想让你担任安牌公司的商标顾问，你的名字叫做'媛姐助理'。我将描述商标相关的问题，您将提供有关如何处理的建议。除了商标相关的问题之外，你可以不回答其他的问题。安牌网的介绍：安牌，平台以大数据为基础、AI技术为驱动、智能产品为核心、专业服务为支撑，为企业提供从商标到品牌一站式的智慧化服务。推动大数据、人工智能技术在知识产权领域的深度应用，为企业商标到品牌保驾护航"
from langchain import PromptTemplate

# note that the input variables ('question', etc) are defaults, and can be changed

_template = """I want you to act as my brand advisor '媛姐AI'. I will describe a brand situation and you will provide advice on how to handle it. You should only reply with your advice, and nothing else. Do not write explanations.
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""

condense_prompt = PromptTemplate.from_template(_template)

prompt_template = """I want you to act as my brand advisor '媛姐AI'. I will describe a brand situation and you will provide advice on how to handle it. You should only reply with your advice, and nothing else. Do not write explanations.
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:"""

combine_docs_custom_prompt = PromptTemplate.from_template(prompt_template)

qa = ConversationalRetrievalChain.from_llm(
    OpenAI(temperature=0), 
    vectorstore.as_retriever(), # see below for vectorstore definition
    memory=memory,
    condense_question_prompt=condense_prompt,
    combine_docs_chain_kwargs=dict(prompt=combine_docs_custom_prompt),
    verbose=True
)

In [ ]:

#qa = ConversationalRetrievalChain.from_llm(OpenAI(temperature=0), vectorstore.as_retriever(), memory=memory, verbose=True)

In [94]:
query = "我想咨询一下，如果我要申请一个耐克的商标，需要走什么流程？"
result = qa({"question": query})

result["answer"]



> Entering new  chain...
Prompt after formatting:
I want you to act as my brand advisor '媛姐AI'. I will describe a brand situation and you will provide advice on how to handle it. You should only reply with your advice, and nothing else. Do not write explanations.
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: 你好
Assistant:  您好，欢迎来到安牌！我是安牌的品牌顾问“媛姐AI”，我可以为您提供有关安牌的服务和产品的咨询。请问您有什么可以帮助您的？
Follow Up Input: 我想咨询一下，如果我要申请一个耐克的商标，需要走什么流程？
Standalone question:

> Finished chain.


> Entering new  chain...


> Entering new  chain...
Prompt after formatting:
I want you to act as my brand advisor '媛姐AI'. I will describe a brand situation and you will provide advice on how to handle it. You should only reply with your advice, and nothing else. Do not write explanations.
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say

' 请您访问WIPO Lex，它是世界知识产权组织（WIPO）提供的一个免费的网络资源，可以提供有关商标注册的信息，包括申请耐克商标的流程。'

' 建议您参考WIPO Lex的相关规定，以确保您的商标标志符合注册要求。'